In [1]:
import glob

import h5py
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# print the list of paths of files in the test set
set_path = "test"
paths = sorted(glob.glob(f"data/{set_path}/*.hdf5"))
print(paths)

[]


In [3]:
# select the first path (arbitrary choice)
p = paths[0]

# print the first layer of keys
with h5py.File(p, "r") as f:
    print(f["dimensions"].keys())
# This dataset is in angular coordinates, so the dimensions are phi and theta.

IndexError: list index out of range

In [ ]:
# In 'boundary_conditions' is stored the information about the boundary conditions:
with h5py.File(p, "r") as f:
    print("print bc available:", f["boundary_conditions"].keys())
    print("print attributes of the bc:",f["boundary_conditions"]["phi_periodic"].attrs["associated_dims"])
    print("get the bc type:", f["boundary_conditions"]["theta_open"].attrs["bc_type"])

In [ ]:
with h5py.File(p, "r") as f:
    print("t0_fields:", f["dimensions"].keys())

In [ ]:
# Reminder: 't0_fields', 't1_fields', 't2_fields' are respectively scalar fields, vector fields and tensor fields
# print the different fields available in the dataset
with h5py.File(p, "r") as f:
    print("t0_fields:", f["t0_fields"].keys())
    print("t1_fields:", f["t1_fields"].keys())
    print("t2_fields:", f["t2_fields"].keys())

In [ ]:
# The data is of shape (n_trajectories, n_timesteps, x, y)
# Get the first t0_field and save it as a numpy array
with h5py.File(p, "r") as f:
    velocity = f["t1_fields"]["velocity"][:]  # HDF5 datasets can be sliced like a numpy array
    print("shape of the selected t0_field: ", velocity.shape)

    # you can directly slice the selected field without reading the whole dataset by doing:
    # traj = 0 #select the trajectory
    # field = f['t0_fields']['concentration'][traj, :]

In [ ]:
traj = 0  # select the trajectory
traj_toplot = velocity[traj, :, :, :, 0]
# field is now of shape (n_timesteps, x, y). Let's do a subplot to plot it at t= 0, t= T/3, t= 2T/3 and t= T:
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
T = traj_toplot.shape[0]

# same colorbar for all subplots:
normalize_plots = False
cmap = "RdBu_r"

if normalize_plots:
    vmin = np.nanmin(traj_toplot)
    vmax = np.nanmax(traj_toplot)
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    for i, t in enumerate([0, T // 3, (2 * T) // 3, T - 1]):
        axs[i].imshow(traj_toplot[t], cmap=cmap, norm=norm)
        axs[i].set_xticks([])
        axs[i].set_yticks([])
        axs[i].set_title(f"t={t}")
else:
    for i, t in enumerate([0, T // 3, (2 * T) // 3, T - 1]):
        axs[i].imshow(traj_toplot[t], cmap=cmap)
        axs[i].set_xticks([])
        axs[i].set_yticks([])
        axs[i].set_title(f"t={t}")
plt.tight_layout()